### Show quantile estimation with SGD  "works"

1. Test datasets:
    - Mixsture of 5 Gaussians
    - Sample of 100k from no.1
    - Gausian $\mu = 2, \sigma = 18$
    - Gausian $\mu = 0, \sigma = 0.001$
    - Sample 100 from no.1
    - Sample 100 from no.3
    - Exponential $\lambda$
    - My weird distribution lol
    
4. Test times:
    - 10 $\times$ dataset generation
    - 10 $\times$ SGD ???
    - 10 $\times$ shuffle (does the order matter?)
    
2. Q value: 0.1, 0.3, 0.5, 0.9, 0.99 

3. Step size: 
    - $\alpha_k = 1$
    - $\alpha_k = \frac{2}{\sqrt{k}}$
    - $\alpha_k = \frac{0.002}{\sqrt{k}}$
    - ?
    - ?
    - Why's the Newton method helpful?
       
5. Calculate $E = |q_{batch} - q_{sgd}|$

6. What's a small value of $E$?

7. Investigate the effect of datasize N
    - Convergence rate with same $\alpha_k$?
    - Different Q values?
    - ?

In [47]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [23]:
q_values = [0.1, 0.3, 0.5, 0.9, 0.99]
N_generation = 12
N_shuffle = 10

a = np.zeros((N_shuffle, len(q_values)))
a.shape

(10, 5)

In [42]:
def generate_dataset(distro, data_size):
    return np.random.normal(0, 0.1, (3,4))

# generate_dataset(True, True)

array([[-0.07310707,  0.22070778, -0.28509463, -0.05076965],
       [ 0.04423327, -0.1456886 , -0.10179877,  0.11275124],
       [ 0.00741648,  0.2047395 , -0.08450196,  0.13094117]])

In [37]:
def generate_q_batch(dataset):
    return np.random.normal(0, 1, len(q_values))

In [38]:
def generate_q_sgd(dataset, step_size):
    return np.random.uniform(0, 2, len(q_values))

In [45]:
def draw_image(q_batches, q_sgds):
#     draw only one line of q_batches, and one block of q_sgds
    print ("-------- test draw_images()-----------")
    
    # how to set the beginning and the end?
    beginning, end = -5, 5
    q_batch_one_gen = q_batches[0]
    
    print (q_batch_one_gen)
    
    print ("--------------- done -----------------")
    return None



In [46]:
def compare_distributions(distro, data_size, step_size):
    q_batches = np.zeros((N_generation, len(q_values)))
    q_sgds = np.zeros((N_generation, N_shuffle, len(q_values)))
    
    # generate datasets
    for i in range(N_generation):
        dataset = generate_dataset(distro, data_size)
        q_batch = generate_q_batch(dataset)
        q_batches[i] = q_batch

        # shuffle
        for j in range(10):
            np.random.shuffle(dataset)
            q_sgd   = generate_q_sgd(dataset, step_size)
            q_sgds[i][j] = q_sgd
    draw_image(q_batches, q_sgds)
#     print (q_batches)
    return None

compare_distributions(True, True, False)

-------- test draw_images()-----------
[ 0.63413083  0.30355086 -1.95992722  1.96469575  0.32964549]
--------------- done -----------------


In [11]:
def compare_step_sizes(step_size, distro, data_size):
    return

In [12]:
def compare_data_size(data_size, distro, setp_size):
    return